In [1]:
import sys
sys.path.append('../..')
import config_neuralnet_stage1

from modules.dataset_utils.bdd_dataset_utils.constants import _IMG_D_, _IMG_RESIZED_H_, _IMG_RESIZED_W_
from modules.pretrained.utils_backbone_cfg import (
    extract_backbone_layers,
    freeze_all_layers,
    get_feat_shapes, 
    extract_fpn_featmap_height_and_width)

In [2]:
basenet = config_neuralnet_stage1.basenet
img_h = _IMG_RESIZED_H_
img_w = _IMG_RESIZED_W_
img_d = _IMG_D_
required_num_return_nodes = config_neuralnet_stage1.num_backbone_nodes
num_extra_blocks_fpn = config_neuralnet_stage1.num_extra_blocks
fpn_hidden_dim = config_neuralnet_stage1.extra_blocks_feat_dim

In [3]:
backbone_feat_shapes = get_feat_shapes(
    basenet, 
    (img_h, img_w, img_d), 
    required_num_return_nodes, 
    num_extra_blocks_fpn, 
    fpn_hidden_dim)

In [4]:
fpn_feat_h, fpn_feat_w = extract_fpn_featmap_height_and_width(backbone_feat_shapes)

In [5]:
backbone_feat_shapes

{'c0': (32, 90, 160),
 'c1': (56, 45, 80),
 'c2': (160, 23, 40),
 'c3': (448, 12, 20),
 'c4': (512, 6, 10)}

In [6]:
fpn_feat_h

{'c0': 90, 'c1': 45, 'c2': 23, 'c3': 12, 'c4': 6}

In [7]:
fpn_feat_w

{'c0': 160, 'c1': 80, 'c2': 40, 'c3': 20, 'c4': 10}

In [8]:
backbone = extract_backbone_layers(basenet, required_num_return_nodes)
backbone = freeze_all_layers(backbone)
backbone

EfficientNet(
  (features): Module(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): SiLU(inplace=True)
    )
    (1): Module(
      (0): Module(
        (block): Module(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivation(
   

In [9]:
import torch 
backbone = extract_backbone_layers(basenet, required_num_return_nodes)
backbone = freeze_all_layers(backbone)

input_data_shape = (1, img_d, img_h, img_w)   # (num batches, num_channels, height, width)
dummy_in = torch.randn(input_data_shape)
dummy_out = backbone(dummy_in)

for key, val in dummy_out.items():
    print(f"block: {key},  shape: {val.shape}")

block: c0,  shape: torch.Size([1, 32, 90, 160])
block: c1,  shape: torch.Size([1, 56, 45, 80])
block: c2,  shape: torch.Size([1, 160, 23, 40])
block: c3,  shape: torch.Size([1, 448, 12, 20])
